In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

In [23]:
from costs import *
from functions import *
from helpers import *

## Load the training data into feature matrix, class labels, and event ids:

In [31]:
from proj1_helpers import *
DATA_TRAIN_PATH = 'train.csv' # TODO: download train data and supply path here 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH, sub_sample=True)

In [32]:
print(y.shape)
head = range(10)
print(y[head])
print(tX[ range(2) ])
print(ids[head])

(5000,)
[ 1. -1. -1.  1.  1.  1. -1. -1.  1. -1.]
[[  1.38470000e+02   5.16550000e+01   9.78270000e+01   2.79800000e+01
    9.10000000e-01   1.24711000e+02   2.66600000e+00   3.06400000e+00
    4.19280000e+01   1.97760000e+02   1.58200000e+00   1.39600000e+00
    2.00000000e-01   3.26380000e+01   1.01700000e+00   3.81000000e-01
    5.16260000e+01   2.27300000e+00  -2.41400000e+00   1.68240000e+01
   -2.77000000e-01   2.58733000e+02   2.00000000e+00   6.74350000e+01
    2.15000000e+00   4.44000000e-01   4.60620000e+01   1.24000000e+00
   -2.47500000e+00   1.13497000e+02]
 [  2.19057000e+02   7.24610000e+01   1.24835000e+02   5.50600000e+00
   -9.99000000e+02  -9.99000000e+02  -9.99000000e+02   3.77100000e+00
    4.69360000e+01   1.22986000e+02   1.93200000e+00  -1.38200000e+00
   -9.99000000e+02   2.47590000e+01  -1.06300000e+00   3.32000000e-01
    4.78300000e+01   1.34700000e+00  -2.56900000e+00   2.84990000e+01
    9.60000000e-01   9.03550000e+01   1.00000000e+00   5.03960000e+01
   

## Do your thing crazy machine learning thing here :) ...

Let's begin with a simple linear regression with least_square gradient descent

In [33]:
gamma = 1.0
max_iter = 10
loss, w = least_squares_SGD(y, tX, gamma, max_iter)

Gradient Descent(0/9): loss=1.0, w0=57.80285279999999, w1=-14.487160399999999
Gradient Descent(1/9): loss=1115219.0460117327, w0=-62710489.51710931, w1=25606519.608812366
Gradient Descent(2/9): loss=3177000159991.1763, w0=168756041635166.22, w1=-72671335396127.05
Gradient Descent(3/9): loss=9.052188135813353e+18, w0=-4.796405845402536e+20, w1=2.070394363938949e+20
Gradient Descent(4/9): loss=2.579231201482503e+25, w0=1.366444326876951e+27, w1=-5.89913322506806e+26
Gradient Descent(5/9): loss=7.3489785341655975e+31, w0=-3.8933581745766207e+33, w1=1.6808340549830848e+33
Gradient Descent(6/9): loss=2.0939373510571364e+38, w0=1.1093299077739035e+40, w1=-4.7891841802214564e+39
Gradient Descent(7/9): loss=5.966235456763936e+44, w0=-3.160802775852641e+46, w1=1.3645776187755787e+46
Gradient Descent(8/9): loss=1.6999537024150425e+51, w0=9.006044749720856e+52, w1=-3.888077820493069e+52
Gradient Descent(9/9): loss=4.843661654483296e+57, w0=-2.566083625081383e+59, w1=1.1078262556160841e+59


In [34]:
print(len(w))
print(loss)

11
[1.0, 1115219.0460117327, 3177000159991.1763, 9.0521881358133535e+18, 2.5792312014825031e+25, 7.3489785341655975e+31, 2.0939373510571364e+38, 5.9662354567639363e+44, 1.6999537024150425e+51, 4.8436616544832959e+57]


Let's try some cross-validation, same as the HW4

In [37]:
from costs import *
from build_polynomial import build_poly

def cross_validation(y, x, k_indices, k, lambda_, degree):
    """return the loss of ridge regression."""
    # ***************************************************
    # INSERT YOUR CODE HERE
    # get k'th subgroup in test, others in train: TODO
    # ***************************************************
    ytest = y[k_indices[k]]
    
    not_k = np.array([i for i in range(len(x)) if i not in k_indices[k]])
    ytrain = y[not_k]
    xtest = x[k_indices[k]]
    xtrain = x[not_k]
    # ***************************************************
    # INSERT YOUR CODE HERE
    # form data with polynomial degree: TODO
    # ***************************************************
    txtrain = build_poly(xtrain, degree)
    txtest = build_poly(xtest, degree)
    # ***************************************************
    # INSERT YOUR CODE HERE
    # ridge regression: TODO
    # ***************************************************
    w = ridge_regression(ytrain, txtrain, lambda_)
    # ***************************************************
    # INSERT YOUR CODE HERE
    # calculate the loss for train and test data: TODO
    # ***************************************************
    loss_tr = compute_rmse(ytrain, txtrain, w)
    loss_te = compute_rmse(ytest, txtest, w)
    

    return loss_tr, loss_te

In [40]:
from plots import cross_validation_visualization
from build_k_indices import *

def cross_validation_demo():
    seed = 1
    degree = 7
    k_fold = 4
    lambdas = np.logspace(-4, 2, 30)
    # split data in k fold
    k_indices = build_k_indices(y, k_fold, seed)
    # define lists to store the loss of training data and test data
    mse_tr = []
    mse_te = []
    # ***************************************************
    # INSERT YOUR CODE HERE
    # cross validation: TODO
    # *************************************************** 
    for lambda_ in lambdas:
        trainerrors = []
        testerrors = []
        for k in range(k_fold):
            trainerror, testerror = cross_validation(y, x, k_indices, k, lambda_, degree)
            trainerrors.append(trainerror)
            testerrors.append(testerror)
        
        mse_tr.append(np.mean(trainerrors))
        mse_te.append(np.mean(testerrors))
    
    
    cross_validation_visualization(lambdas, mse_tr, mse_te)

cross_validation_demo()

NameError: name 'x' is not defined

## Generate predictions and save ouput in csv format for submission:

In [10]:
DATA_TEST_PATH = '' # TODO: download train data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [31]:
OUTPUT_PATH = '' # TODO: fill in desired name of output file for submission
y_pred = predict_labels(weights, tX_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)